<a href="https://colab.research.google.com/github/prasanth5reddy/D2L/blob/master/Modern%20Convolutional%20Networks/deep_cnn_AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
w_dir = '/content/drive/My Drive/Colab/D2L.AI/'
sys.path.append(w_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installing Libraries

In [2]:
!pip uninstall mxnet && pip install mxnet-cu100

Importing Libraries

In [0]:
from mxnet import gluon, init, nd
from mxnet.gluon import data as gdata, nn
import d2l
import os
import sys

AlexNet

In [0]:
net = nn.Sequential()
net.add(nn.Conv2D(96, kernel_size=11, strides=4, activation='relu'),
        nn.MaxPool2D(pool_size=3, strides=2),
        nn.Conv2D(256, kernel_size=5, padding=2, activation='relu'),
        nn.MaxPool2D(pool_size=3, strides=2),
        nn.Conv2D(384, kernel_size=3, padding=1, activation='relu'),
        nn.Conv2D(384, kernel_size=3, padding=1, activation='relu'),
        nn.Conv2D(256, kernel_size=3, padding=1, activation='relu'),
        nn.MaxPool2D(pool_size=3, strides=2),
        # Here, the number of outputs of the fully connected layer is several times larger than that in LeNet.
        # Use the dropout layer to mitigate overfitting
        nn.Dense(4096, activation='relu'), nn.Dropout(0.5),
        nn.Dense(4096, activation='relu'), nn.Dropout(0.5),
        nn.Dense(10))

In [5]:
X = nd.random.uniform(shape=(1, 1, 224, 224))
net.initialize()
for layer in net:
  X = layer(X)
  print(layer.name, 'output shape:\t', X.shape)

conv0 output shape:	 (1, 96, 54, 54)
pool0 output shape:	 (1, 96, 26, 26)
conv1 output shape:	 (1, 256, 26, 26)
pool1 output shape:	 (1, 256, 12, 12)
conv2 output shape:	 (1, 384, 12, 12)
conv3 output shape:	 (1, 384, 12, 12)
conv4 output shape:	 (1, 256, 12, 12)
pool2 output shape:	 (1, 256, 5, 5)
dense0 output shape:	 (1, 4096)
dropout0 output shape:	 (1, 4096)
dense1 output shape:	 (1, 4096)
dropout1 output shape:	 (1, 4096)
dense2 output shape:	 (1, 10)


Reading Data

In [0]:
def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join('~', '.mxent', 'datasets', 'fashion-mnist')):
  root = os.path.expanduser(root)
  transformer = []
  if resize:
    transformer += [gdata.vision.transforms.Resize(resize)]
  transformer += [gdata.vision.transforms.ToTensor()]
  transformer = gdata.vision.transforms.Compose(transformer)
  mnist_train = gdata.vision.FashionMNIST(root=root, train=True)
  mnist_test = gdata.vision.FashionMNIST(root=root, train=False)
  num_workers = 0 if sys.platform.startswith('win32') else 4
  train_iter = gdata.DataLoader(mnist_train.transform_first(transformer), batch_size, shuffle=True, num_workers=num_workers)
  test_iter = gdata.DataLoader(mnist_train.transform_first(transformer), batch_size, shuffle=True, num_workers=num_workers)
  
  return train_iter, test_iter

In [0]:
batch_size = 128
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

Training

In [8]:
lr, num_epochs, ctx = 0.01, 5, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 1.3059, train acc 0.509, test acc 0.761, time 72.1 sec
epoch 2, loss 0.6478, train acc 0.759, test acc 0.810, time 69.3 sec
epoch 3, loss 0.5318, train acc 0.802, test acc 0.842, time 69.2 sec
epoch 4, loss 0.4656, train acc 0.829, test acc 0.859, time 70.4 sec
epoch 5, loss 0.4236, train acc 0.846, test acc 0.867, time 70.0 sec
